# Flask

É um framework que contém sua própria coleção de módulos para desenvolvimento de aplicações a partir de regras e formatos pré-definidos. No caso do Flask, fica a cargo do desenvolvedor importar novos pacotes e funcionalidades criadas por outros usuários para compor o site. 

O Flask pode ser caracterizado como "micro-framework full-stack" por possuir todos os requisitos para desenvolvimento de aplicações web, desde a definição do template do site até a ligação com os bancos de dados que interagem com as aplicações e fornecem informações.

#### Sintaxe Básica

In [7]:
from flask import Flask

app = Flask(__name__)

@app.route("/") # cria uma página para o site
def home():
    return "Página Inicial" # HTML da página

if __name__ == "__main__":
    app.run() # Roda o código. Setar "debug = True" permite atualizar a página enquanto adicionamos novas funcionalidades.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


#### Interação com Front-end

Os códigos-fonte HTML, CSS e JavaScript estilizam as páginas da aplicação web. O Flask recomenda que arquivos de marcação (HTML), que definem a estrutura das páginas, sejam inseridos dentro do projeto em uma pasta chamada "templates". Os códigos que realizam scripts (JS) e estilizações (CSS) devem ser inseridos em uma pasta denominada "static", em conjunto com os demais arquivos que serão introduzidos de forma "fixa" no projeto.

O Flask também permite inserir parte do código e variáveis criadas em Python direto do back-end para dentro do HTML da página, com a utilização de chaves duplas do tipo `{{}}`. Além disso, também há suporte para o Jinja2, que permite a utilização de outras funcionalidades, como `for` statements, `if` statements e o `block`, utilizando {% %}. 

É dado destaque aos blocks, pois permitem a herança de código entre templates, ou seja, utilizar um mesmo template como base para a criação de outros, preenchendo lacunas predefinidas dele (ou blocks). Segue abaixo o exemplo:

```
# Arquivo base.html
<!doctype html>
<html>
  <head>
    {% block head %}
    **INSIRA O CÓDIGO AQUI**
    {% endblock %}
  </head>
  <body>
    <div id="content">{% block content %}{% endblock %}</div>
    <div id="footer">
      {% block footer %}
      **INSIRA O CODIGO AQUI**
      {% endblock %}
    </div>
  </body>
</html>

# Arquivo página.html

{% extends "base.html" %}

{% block main %}
**PREENCHE MAIN**
{% endblock %}

{% block footer %}
**PREENCHE FOOTER**
{% endblock %}
```

#### Sessions

As sessões são iniciadas a partir do momento em que acessamos o site em determinado navegador, e finalizadas quando saímos dele. Sessions permitem armazenar variáveis obtidas a partir do momento em que o usuário interagiu com determinado template, e utilizar as informações em outros pontos do site. Exemplo:

```
@app.route("/login")
def login():
    if request.method == "POST":
        email = request.form["email"]
        senha = request.form["senha"]

        login_existente = users.query.filter_by(email = email, senha = senha).first()
        if login_existente:
            session["email"] = email
            session["senha"] = senha
            return redirect(url_for("admin"))                       
        else:
            flash("E-mail ou senha incorretos!", "info")
            return render_template("login.html")
            
    else: # caso em que request.method = "GET"
            return render_template("login.html")

@app.route("/login/admin") 
def admin(): # só pode acessar se já estiver logado
    if "email" in session and "senha" in session:
        email = session["email"]
        senha = session["senha"]
        return render_template("admin.html")
    else:
        return redirect(url_for("login.html"))
```

Para manipular sessions, é necessário definir uma chave secreta para a criptografia dos dados armazenados, conforme exemplo abaixo:

```app.secret_key = "key"```

#### Flash Messages

São mensagens que fornecem feedback ao usuário caso interaja com o site em certas situações. Podem ser criadas utilizando a função `flash` no Python, e acessadas como lista no template HTML com o comando `{% with messages in get_flashed_messages() %}`.

Exemplo:
```
{% with messages = get_flashed_messages() %}
  {% if messages %}
    {% for message in messages}
        <p>{{message}} </p>
    {% endfor %}
  {% endif %}
{% endwith %}

```

#### SQLAlchemy e integração com SQLite3

O Flask possui um ORM (Object Relationship Mapper) próprio denominado `flask_sqlalchemy`, que permite a realização de consultas a bancos de dados de maneira bem simples, com Programação Orientada a Objetos. Por padrão, o ORM utiliza conexões com o SQLite3, que já possui suporte para o Python. Para configurar a base de dados que armazenará nossas tabelas, é necessária a definição de certos parâmetros, como:



```
from flask_sqlalchemy import SQLAlchemy

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///database.sqlite3"
db = SQLAlchemy(app)

with app.app_context():
        db.create_all()
```

Com o SQLAlchemy é possível criar um banco de dados do tipo .db dentro da pasta do projeto e acessá-lo de maneira simples. Cada tabela dentro do banco de dados é definida como uma classe, e as suas características (colunas) são atributos de classe. As linhas da tabela são acessadas como instâncias dessa classe, e possuem características (colunas) que podem ser acessadas conforme o exemplo:


```
class users(db.Model): # cria tabela users
    
    id = db.Column("id", db.Integer, primary_key = True)
    email = db.Column("email", db.String(100))
    senha = db.Column("senha", db.String(100))

    def __init__(self, email, senha):
        self.email = email
        self.senha = senha

@app.route("/login/admin/lista_admins")
def lista_admins(): # Realiza um GET na base de dados
    return render_template("lista_admins.html", values = users.query.all()) # cria uma lista com objetos da classe Users e passa como variável para o template

# No template HTML...
{% for usuario in values %}
    <p>E-mail: {{usuario.email}}, Senha: {{usuario.senha}}</p> 
{% endfor %}


```